In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58889256/58889256 [==============================] - 1s 0us/step


In [2]:
base_model.trainable = False

In [3]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(6, activation = 'softmax')(x)

model = keras.Model(inputs, outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 6)                 3078      
                                                                 
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])

In [6]:
#Augment the Data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)

In [9]:
#LOAD DATASET
train_it = datagen.flow_from_directory("data/fruits/train/", 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")

valid_it = datagen.flow_from_directory("data/fruits/valid/", 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

FileNotFoundError: ignored

In [ ]:
#TRAIN THE MODEL
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=30)

In [ ]:
# Unfreeze the base model
base_model.trainable = True
# Compile the model with a low learning rate
model.compile(loss = "categorical_crossentropy",
              optimizer = keras.optimizers.RMSprop(learning_rate = 0.00005),
              metrics = ['accuracy'])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=45,
          initial_epoch=30)

In [ ]:
#EVALUATE THE MODEL
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)